In [ ]:
# Standard libraries of Python

from time import gmtime

# Dependencies
import pandas as pd
import requests
import re
from pandas import DataFrame
from bs4 import BeautifulSoup

from backend.database.src.scraping import eight_gen
from backend.database.src import parse

In [ ]:
pokedex = pd.DataFrame()

In [ ]:
assert requests.get("https://www.serebii.net/index2.shtml").status_code == 200, 'There is a problem with Euromillions webpage'

In [ ]:
df = pd.read_html('https://www.serebii.net/pokedex-sm/001.shtml')

In [ ]:
name = parse.identity(df)
gender = parse.gender(df)
weight = parse.weight(df)
hab = parse.hab(df)
weaknesess = parse.weaknesses(df)
egg_group = parse.egg_group(df)

In [ ]:
weaknesess

In [ ]:
pd.concat([name,gender,weight],axis=1).reset_index(drop=True)

In [ ]:
pd.concat([patata,hab],axis=1)

In [ ]:
df[0]

In [ ]:
data = df.copy()

In [ ]:
data[23]

In [ ]:
abilities = df[9]
criteria = '. Hidden Ability (Available through transfer): '
abilities = abilities.loc[1,1].split(criteria)
hab_df = pd.DataFrame([
    {'Ability': abilities[0], 'Hidden Ability': abilities[1]}
])

In [ ]:
hab_df

In [ ]:
abilities[0]